In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

from collections import Counter
import operator

In [2]:
filename = '../data/2019-Oct_.csv_10%.csv'

In [3]:
import random
p = 1  # 1% of the lines
# keep the header, then take only 1% of lines
# if random from [0,1] interval is greater than 0.01 the row will be skipped
df = pd.read_csv(
         filename,
         header=0, 
         skiprows=lambda i: i>0 and random.random() > p
)
df.shape

FileNotFoundError: [Errno 2] No such file or directory: '../data/2019-Oct_.csv_10%.csv'

# Preprocess

In [ ]:
def preprocessing_feat(X, drop_event_time=False):
    if drop_event_time:
        X = X.drop("event_time", axis=1)
    
    X_preprocessed = X.dropna(subset = ['category_code', 'brand']) #tbd!!
    X_preprocessed = X_preprocessed.drop_duplicates()
    X_preprocessed['category_code'] = X_preprocessed['category_code'].str.replace('.',' ')
    return X_preprocessed

In [ ]:
X_preprocessed = preprocessing_feat(df, drop_event_time=False)

In [ ]:
X_preprocessed.shape

In [ ]:
def make_column(row):
    if row["price"] < row["25%"]:
        return "low"
    elif row["price"] < row["75%"]:
        return "medium"
    else:
        return "high"

def pricing_criterion(X):
    pricing_guide = X.groupby('category_code')['price'].describe()[["25%", "75%"]].reset_index()
    X_merged = X.merge(pricing_guide, on="category_code", how="right")
    X_merged["price_category"] = X_merged.apply(lambda row: make_column(row), axis=1)
    return X_merged


In [ ]:
X_merged = pricing_criterion(X_preprocessed)

In [ ]:
def metadata(X):
    X['metadata'] = X[['category_code', 'brand', 'price_category']].apply(lambda x: ' '.join(x), axis = 1)
    return X


In [ ]:
X_meta = metadata(X_merged)
X_meta.shape

In [ ]:
X_meta.head()

In [ ]:
X_purch = X_meta[X_meta['event_type']=='purchase']

In [ ]:
X_purch.shape

In [ ]:
def filter(X):
    df = pd.DataFrame(X.groupby('user_session').event_type.count())
    df_new = df[df['event_type']>1]
    return df_new

In [ ]:
X_filter = filter(X_purch)

In [ ]:
X_filter.shape

In [ ]:
X_filter.head()

In [ ]:
df_cross = X_purch[X_purch['user_session'].isin(X_filter.index)]

In [ ]:
df_cross.shape

# Counter

In [ ]:
df_test = pd.DataFrame(df_cross.groupby('user_session')['product_id'].apply(list).tolist())
df_test.head()

In [ ]:
def concat(X):
    X = X[X[0]!=X[1]]
    X['conc'] = X[0].astype(str) + '_' + X[1].astype(str)
    
    lst = list(X['conc'])
    
    a = list(Counter(lst).keys()) # equals to list(X['conc'])
    b = list(Counter(lst).values()) # counts the elements' frequency
    
    dct_1 = {a[i]: b[i] for i in range(len(a))}
    dct_2 = dict(sorted(dct_1.items(), key=operator.itemgetter(1),reverse=True))

    return pd.DataFrame.from_dict(dct_2, orient='index').reset_index().rename(columns={'index': 'combo', 0: 'counts'})


In [ ]:
X_sell = concat(df_test)

In [ ]:
X_sell

In [ ]:
def x_seller(X):
    X["product_1"] = X["combo"].str.split("_",expand=True)[0].astype('category')
    X["product_2"] = X["combo"].str.split("_",expand=True)[1].astype('category')
    return X
    

In [ ]:
new_dataframe = x_seller(X_sell)

new_dataframe.product_1 = new_dataframe.product_1.apply(lambda x: int(x))
new_dataframe.product_2 = new_dataframe.product_2.apply(lambda x: int(x))


In [ ]:
new_dataframe.shape

In [ ]:
new_dataframe.head()

In [ ]:
new_df_1 = new_dataframe.merge(X_meta, how='left', left_on='product_1', right_on='product_id').rename(columns={'price': 'price_1', 'metadata': 'metadata_1'})

In [ ]:
new_df_1.shape

In [ ]:
new_df_1 = new_df_1.drop_duplicates(['product_1','product_2'])
new_df_1.shape

In [ ]:
new_df_1.head()

In [ ]:
df_short = new_df_1[['combo', 'counts', 'product_1', 'product_2', 'price_1', 'metadata_1']]
df_short.head()

In [ ]:
df_short.shape

In [ ]:
df_final = df_short.merge(X_meta, how='left', left_on='product_2', right_on='product_id').rename(columns={'price': 'price_2', 'metadata': 'metadata_2'}).drop_duplicates(['product_1','product_2'])

In [ ]:
df_final.shape

# Final recommendation

In [ ]:
df_final = df_final[['combo', 'counts', 'product_1', 'product_2', 'price_1', 'price_2', 'metadata_1', 'metadata_2']]

In [ ]:
df_final.shape

In [ ]:
df_final.head()

In [ ]:
df_final = df_final[df_final['metadata_1'] != df_final['metadata_2']]

In [ ]:
df_final.head()

In [ ]:
df_final["1.1"] = df_final["metadata_1"].str.split(" ",expand=True)[0].astype('category')
df_final["1.2"] = df_final["metadata_1"].str.split(" ",expand=True)[1].astype('category')
df_final.head()

In [ ]:
df_final["2.1"] = df_final["metadata_2"].str.split(" ",expand=True)[0].astype('category')
df_final["2.2"] = df_final["metadata_2"].str.split(" ",expand=True)[1].astype('category')
df_final.shape

In [ ]:
df_final['test_1'] = df_final[['1.1', '1.2']].apply(lambda x: ' '.join(x), axis = 1)
df_final['test_2'] = df_final[['2.1', '2.2']].apply(lambda x: ' '.join(x), axis = 1)

In [ ]:
df_final.head()

In [ ]:
df_final_2 = df_final[df_final['test_1'] != df_final['test_2']]

In [ ]:
df_final_2.shape

In [ ]:
df_final_2.head()